In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [26]:

file_path = r'C:\Users\ematrejek\Downloads\wyniki_wyborow.xlsx' #zamień na Twoją ścieżkę z plikiem

data = pd.read_excel(file_path)

# Usuń wiersze z brakującymi wartościami
data_cleaned = data.dropna()

# Przekształć dane na pivot table
pivot_data = data_cleaned.pivot_table(index='Kod TERYT', columns=['wybory', 'partia'], values='głosy', fill_value=0)

# Spłaszcz kolumny
pivot_data.columns = ['_'.join(col).strip() for col in pivot_data.columns.values]
pivot_data.reset_index(inplace=True)



In [29]:
# Przygotowanie danych historycznych
# Zakładam, że mamy dane z wyborów prezydenckich 2020, do sejmu 2019 i 2023
X = pivot_data.drop(columns=['Kod TERYT'])
y_columns = [col for col in pivot_data.columns if 'do sejmu 2023' in col]

In [30]:
# Przewidywanie wyników dla każdej partii w wyborach do sejmu 2023
results = {}

for y_col in y_columns:
    # Podziel dane na cechy (X) i etykiety (y)
    y = pivot_data[y_col]

    # Podziel dane na zestaw treningowy i testowy
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Stwórz i naucz model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Przewidywanie wyników
    y_pred = model.predict(X_test)

    # Ocena modelu
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Mean Absolute Error for {y_col}: {mae}')

    # Przewidywanie na nowych danych (wybory do Parlamentu Europejskiego 2024)
    y_pred_new = model.predict(X)

    # Zapisz wyniki
    results[y_col] = y_pred_new.mean()  # Zapisz średnią z przewidywanych wyników

# Przedstawienie wyników
print("Predicted election results:")

# Sortowanie wyników i wyświetlanie w kolejności od najwyższego do najniższego
sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
for rank, (party, prediction) in enumerate(sorted_results, 1):
    print(f"{rank}. {party}: {prediction}")

# Oblicz procentowe wyniki
total_votes = sum(results.values())
percentage_results = {party: (votes / total_votes) * 100 for party, votes in results.items()}

print("\nPercentage results:")
for party, percentage in percentage_results.items():
    print(f"{party}: {percentage:.2f}%")

# Opis metody
method_description = """
Użyty model: Regresja liniowa
Zmienne: Wyniki wyborów prezydenckich 2020, wyniki wyborów do sejmu 2019 i 2023
"""

print("Description of the method:")
print(method_description)

Mean Absolute Error for do sejmu 2023_KONFEDERACJA: 8.86384299292331e-14
Mean Absolute Error for do sejmu 2023_LEWICA: 4.8622439408063656e-14
Mean Absolute Error for do sejmu 2023_PIS: 4.885549742539297e-13
Mean Absolute Error for do sejmu 2023_PO: 1.8578560911919338e-13
Mean Absolute Error for do sejmu 2023_TRZECIA DROGA: 1.2603607046912657e-13
Predicted election results:
1. do sejmu 2023_PIS: 238.40628419399758
2. do sejmu 2023_PO: 138.20830831326083
3. do sejmu 2023_TRZECIA DROGA: 83.35557212336685
4. do sejmu 2023_KONFEDERACJA: 42.638344441845724
5. do sejmu 2023_LEWICA: 38.0345205018994

Percentage results:
do sejmu 2023_KONFEDERACJA: 7.89%
do sejmu 2023_LEWICA: 7.04%
do sejmu 2023_PIS: 44.10%
do sejmu 2023_PO: 25.56%
do sejmu 2023_TRZECIA DROGA: 15.42%
Description of the method:

Użyty model: Regresja liniowa
Zmienne: Wyniki wyborów prezydenckich 2020, wyniki wyborów do sejmu 2019 i 2023



Z racji, że sytuacja w polityce jest dosyć dynamiczna, przeszukaliśmy artykuły w Internecie w poszukiwaniu wyników tegorocznych sondaży przedwyborczych.
Zgromadziliśmy wyniki 8 różnych sondaży, m.in. CBOS, IBRiS i IPSOS.
Następnie wyliczyliśmy średnie wyniki procentowe 5 głównych partii, które wyglądają następująco:

KONFEDERACJA	9.8%
LEWICA	8.0%
PIS	30.2%
PO	30.3%
TRZECIA_DROGA	10.6%

W oparciu o te wyniki, a także o oszacowania MAE regresji liniowej, dokonujemy poprawki oszacowań regresji liniowej.

In [34]:
# Mean Absolute Error for each party
mae = {
    'KONFEDERACJA': 8.86384299292331e-14,
    'LEWICA': 4.8622439408063656e-14,
    'PIS': 4.885549742539297e-13,
    'PO': 1.8578560911919338e-13,
    'TRZECIA DROGA': 1.2603607046912657e-13
}


# Pre-election survey results in percentages
survey_results = {
    'KONFEDERACJA': 9.8,
    'LEWICA': 8.0,
    'PIS': 30.2,
    'PO': 30.3,
    'TRZECIA DROGA': 10.63
}

# Convert survey percentages to proportions
total_survey = sum(survey_results.values())
survey_proportions = {party: (value / total_survey) for party, value in survey_results.items()}

# Calculate the adjustment factor for each party
total_predicted = sum(predicted_results.values())
predicted_proportions = {party: (value / total_predicted) for party, value in predicted_results.items()}

# Adjust the predicted results based on survey data
adjusted_results = {party: (predicted_results[party] * survey_proportions[party] / predicted_proportions[party]) 
                    for party in predicted_results}

# Calculate the new total and percentages
adjusted_total = sum(adjusted_results.values())
adjusted_percentages = {party: (value / adjusted_total) * 100 for party, value in adjusted_results.items()}

print("\nAdjusted Percentage Results:")
for party, percentage in sorted(adjusted_percentages.items(), key=lambda x: x[1], reverse=True):
    print(f"{party}: {percentage:.2f}%")


Adjusted Percentage Results:
PO: 34.07%
PIS: 33.96%
TRZECIA DROGA: 11.95%
KONFEDERACJA: 11.02%
LEWICA: 9.00%
